# Lesson 4: Chaining Prompts for Agentic Reasoning

## Automated Claim Triage: From First-Notice to the Right Queue

In this hands-on exercise, you will build a prompt chain that extracts key fields from free-form auto-claim reports, assesses damage severity, and routes each claim to one of several queues — `glass`, `fast_track`, `material_damage`, or `total_loss` — with code-based gate checks at every step.

## Outline:

- Setup
- Sample FNOL (First Notice of Loss) Texts
- Stage I: Information Extraction
- Stage II: Severity Assessment
- Stage III: Queue Routing
- Review Outputs

## Setup

Import necessary libraries and define helper functions, including a mock LLM client, code execution environment, and test runner.

In [1]:
# Import necessary libraries
# No changes needed in this cell
from openai import OpenAI  # For accessing the OpenAI API
from enum import Enum
import json
from pydantic import BaseModel, Field  # For structured data validation
from typing import List, Literal, Optional

In [2]:
# Set up LLM credentials

client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    # api_key="**********",  # <--- TODO: Fill in your Vocareum API key here
    api_key=os.getenv(
        "OPENAI_API_KEY"
    ),  # <-- Alternately, set as an environment variable
)

# If using OpenAI's API endpoint
# client = OpenAI()

In [3]:
# Define helper functions
# No changes needed in this cell


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = OpenAIModels.GPT_41_NANO


def get_completion(messages=None, system_prompt=None, user_prompt=None, model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt
        user_prompt: The user prompt
        model: The model to use (default is gpt-4.1-mini)
    Returns:
        The completion text
    """

    messages = list(messages)
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    if user_prompt:
        messages.append({"role": "user", "content": user_prompt})
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
    )
    return response.choices[0].message.content

## Sample FNOL (First Notice of Loss) Texts
Let's define a few sample First Notice of Loss (FNOL) texts to process through our chain.

In [4]:
# Define sample FNOL texts
# TODO: [Optional] Add more sample FNOL texts to test various scenarios

sample_fnols = [
    """
    Claim ID: C001
    Customer: John Smith
    Vehicle: 2018 Toyota Camry
    Incident: While driving on the highway, a rock hit my windshield and caused a small chip
    about the size of a quarter. No other damage was observed.
    """,
    """
    Claim ID: C002
    Customer: Sarah Johnson
    Vehicle: 2020 Honda Civic
    Incident: I was parked at the grocery store and returned to find someone had hit my car and
    dented the rear bumper and taillight. The taillight is broken and the bumper has a large dent.
    """,
    """
    Claim ID: C003
    Customer: Michael Rodriguez
    Vehicle: 2022 Ford F-150
    Incident: I was involved in a serious collision at an intersection. The front of my truck is
    severely damaged, including the hood, bumper, radiator, and engine compartment. The airbags
    deployed and the vehicle is not drivable.
    """,
    """
    Claim ID: C004
    Customer: Emma Williams
    Vehicle: 2019 Subaru Outback
    Incident: My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and
    trunk. The side mirrors were also damaged and one window has a small crack.
    """,
    """
    Claim ID: C005
    Customer: David Brown
    Vehicle: 2021 Tesla Model 3
    Incident: Someone keyed my car in the parking lot. There are deep scratches along both doors
    on the driver's side.
    """,
    """"
    Claim ID: C006
    Customer: Lisa Davis
    Vehicle: 2017 BMW X5
    Incident: I was rear-ended at a stoplight. The back of my SUV is damaged, including the bumper, trunk, and rear lights. The rear windshield is also shattered.
    """,
    """"
    Claim ID: C007
    Customer: James Wilson
    Vehicle: 2016 Audi A4
    Incident: I hit a deer while driving at night. The front of my car is damaged, including the bumper, grille, and headlights. The hood is also dented and there may be
    damage to the radiator.""",

    """
    Claim ID: C008
    Customer: Ana Pereira
    Vehicle: 2015 Hyundai Elantra
    Incident: While backing out of a parking space, I lightly hit the car behind me. There is a small dent on the rear bumper and a scuff on the paint. The vehicle is fully drivable and no injuries occurred.
    """,

    """
    Claim ID: C009
    Customer: Bruno Silva
    Vehicle: 2012 Volkswagen Golf
    Incident: I clipped a curb and scraped the right front quarter panel while making a tight turn. There is a dent and paint damage but the car is drivable and nobody was hurt.
    """,

    """
    Claim ID: C010
    Customer: Carla Mendes
    Vehicle: 2019 Chevrolet Silverado
    Incident: I was involved in a high-speed multi-vehicle collision on the motorway. The front of my truck is heavily damaged, airbags deployed, and two passengers sustained serious injuries and were transported to the hospital. The vehicle is not drivable.
    """,
]


## Stage I: Information Extraction
In this stage, we'll create a prompt that extracts structured information from free-form FNOL text.

In [5]:
# Define a system prompt for information extraction according to the provided ClaimInformation class
# TODO: Complete the prompt by replacing the parts marked with **********


class ClaimInformation(BaseModel):
    claim_id: str = Field(..., min_length=2, max_length=10)
    name: str = Field(..., min_length=2, max_length=100)
    vehicle: str = Field(..., min_length=2, max_length=100)
    loss_desc: str = Field(..., min_length=10, max_length=500)
    damage_area: List[
        Literal[
            "windshield",
            "front",
            "rear",
            "side",
            "roof",
            "hood",
            "door",
            "bumper",
            "fender",
            "quarter panel",
            "trunk",
            "glass",
        ]
    ] = Field(..., min_items=1)


info_extraction_system_prompt = """
You are an expert data extraction AI specializing in insurance claims. Your task is to extract key information from First Notice of Loss (FNOL) reports.

Format your response as a valid JSON object with the following keys:
- claim_id (str): The claim ID
- name (str): The customer's full name
- vehicle (str): The make, model, and year of the vehicle
- loss_desc (str): A description of the loss
- damage_area (List[str]): A list of the areas of the vehicle that were damaged, chosen ONLY from the following options: 
    - windshield
    - front
    - rear
    - side
    - roof
    - hood
    - door
    - bumper
    - fender
    - quarter panel
    - trunk
    - glass



Only respond with the JSON object, nothing else.
"""

In [6]:
# Define a gate check function and claim extraction function
# TODO: Complete the prompt by replacing the parts marked with **********


def gate1_validate_claim_info(claim_info_json: str) -> ClaimInformation:
    """
    Gate 1: Validates claim information extracted from FNOL text.
    Returns validated ClaimInformation object or raises validation error.
    """
    try:
        # Parse the JSON string
        claim_info_dict = json.loads(claim_info_json)
        # Validate with Pydantic model
        validated_info = ClaimInformation(**claim_info_dict)
        return validated_info
    except Exception as e:
        raise ValueError(f"Gate 1 validation failed: {str(e)}")


def extract_claim_info(fnol_text):
    """
    Stage 1: Extract structured information from FNOL text
    """
    messages = [
        {"role": "system", "content": info_extraction_system_prompt},
        {"role": "user", "content": fnol_text},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the extracted information
    try:
        validated_info = gate1_validate_claim_info(response)
        return validated_info
    except ValueError as e:
        print(f"Gate 1 failed: {e}")
        return None

In [7]:
# Run the claim extraction function on the sample FNOLs
# No updates needed in this cell

extracted_claim_info_items = [
    extract_claim_info(fnol_text) for fnol_text in sample_fnols
]
extracted_claim_info_items

[ClaimInformation(claim_id='C001', name='John Smith', vehicle='2018 Toyota Camry', loss_desc='While driving on the highway, a rock hit my windshield and caused a small chip about the size of a quarter. No other damage was observed.', damage_area=['windshield']),
 ClaimInformation(claim_id='C002', name='Sarah Johnson', vehicle='2020 Honda Civic', loss_desc='Someone hit the parked car, denting the rear bumper and breaking the taillight.', damage_area=['rear', 'bumper', 'glass']),
 ClaimInformation(claim_id='C003', name='Michael Rodriguez', vehicle='2022 Ford F-150', loss_desc='Serious collision at an intersection with severe front-end damage, including hood, bumper, radiator, and engine compartment. Airbags deployed and vehicle is not drivable.', damage_area=['front', 'hood', 'bumper']),
 ClaimInformation(claim_id='C004', name='Emma Williams', vehicle='2019 Subaru Outback', loss_desc='My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and trunk. The side mirro

## Stage II: Severity Assessment
In this stage, we'll assess the severity of the damage based on the extracted information.

Note, our carrier applies the following heuristics:
- Minor damage: Small dents, scratches, glass chips (cost range: $100-$1,000)
- Moderate damage: Single panel damage, bumper replacement, door damage (cost range: $1,000-$5,000)
- Major damage: Structural damage, multiple panel replacement, engine/drivetrain issues, total loss candidates (cost range: $5,000-$50,000)

In this example we will let the LLM estimate the cost, though in production we would want a more accurate estimate, e.g. querying a database of repair costs.


In [8]:
# Define a system prompt for severity assessment according to the provided SeverityAssessment class
# TODO: Complete the prompt by replacing the parts marked with **********


class SeverityAssessment(BaseModel):
    severity: Literal["Minor", "Moderate", "Major"]
    est_cost: float = Field(..., gt=0)


severity_assessment_system_prompt = """
You are an auto insurance damage claims adjuster. Your task is to evaluate the severity of vehicle damage and estimate repair costs.

Based on the claim information, particularly the loss_desc and damage_area, classify the damage (severity) as 'Minor', 'Moderate', or 'Major'. 
Also provide an estimated repair cost (est_cost) as a float with no surrounding text. Use the following guidelines for severity and typical costs: 
    - Minor ($100-$1000) 
    - Moderate ($1000-$5000)
    - Major ($5000-$50000)

Only respond with the JSON object, nothing else.
"""

In [9]:
# Define a gate check function and assess_severity function
# TODO: Complete the prompt by replacing the parts marked with **********


def gate2_cost_range_ok(severity_json: str) -> SeverityAssessment:
    """
    Gate 2: Validates that the estimated cost is within reasonable range for the severity.
    Returns validated SeverityAssessment object or raises validation error.
    """
    try:
        # Parse the JSON string
        severity_dict = json.loads(severity_json)
        # Validate with Pydantic model
        validated_severity = SeverityAssessment(**severity_dict)

        # Check cost range based on severity
        if (
            validated_severity.severity == "Minor"
            and (
                validated_severity.est_cost < 100 or validated_severity.est_cost > 1000
            )
        ):
            raise ValueError(
                f"Minor damage should cost between $100-$1000, got ${validated_severity.est_cost}"
            )
        elif (
            validated_severity.severity == "Moderate"
            and (
                validated_severity.est_cost < 1000 or validated_severity.est_cost > 5000
            )
        ):
            raise ValueError(
                f"Moderate damage should cost between $1000-$5000, got ${validated_severity.est_cost}"
            )
        elif (
            validated_severity.severity == "Major"
            and (
                validated_severity.est_cost < 5000 or validated_severity.est_cost > 50000
            )
        ):
            raise ValueError(
                f"Major damage should cost between $5000-$50000, got ${validated_severity.est_cost}"
            )


        return validated_severity
    except Exception as e:
        raise ValueError(f"Gate 2 validation failed: {str(e)}")


def assess_severity(claim_info: ClaimInformation) -> Optional[SeverityAssessment]:
    """
    Stage 2: Assess severity based on damage description
    """

    # Convert Pydantic model to JSON string
    claim_info_json = claim_info.model_dump_json()

    messages = [
        {"role": "system", "content": severity_assessment_system_prompt},
        {"role": "user", "content": claim_info_json},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the severity assessment
    try:
        validated_severity = gate2_cost_range_ok(response)
        return validated_severity
    except ValueError as e:
        print(f"Gate 2 failed: {e}. Response: {response}")
        return None


In [10]:
# Run the claim extraction function on the sample data
# No updates needed in this cell

severity_assessment_items = [
    assess_severity(item) for item in extracted_claim_info_items
]

severity_assessment_items

[SeverityAssessment(severity='Minor', est_cost=150.0),
 SeverityAssessment(severity='Moderate', est_cost=2500.0),
 SeverityAssessment(severity='Major', est_cost=15000.0),
 SeverityAssessment(severity='Moderate', est_cost=2500.0),
 SeverityAssessment(severity='Moderate', est_cost=2500.0),
 SeverityAssessment(severity='Moderate', est_cost=3500.0),
 SeverityAssessment(severity='Major', est_cost=15000.0),
 SeverityAssessment(severity='Minor', est_cost=300.0),
 SeverityAssessment(severity='Minor', est_cost=450.0),
 SeverityAssessment(severity='Major', est_cost=15000.0)]

## 6. Stage III: Queue Routing
In this stage, we'll route the claim to the appropriate queue based on severity and damage area.

Use these routing rules:
- 'glass' queue: For Minor damage involving ONLY glass (windshield, windows)
- 'fast_track' queue: For other Minor damage
- 'material_damage' queue: For all Moderate damage
- 'total_loss' queue: For all Major damage

These are the priority levels:
- Priority 1 (highest): Safety issues, customer stranded
- Priority 2: Significant but contained damage, vehicle drivable
- Priority 3: Standard claims
- Priority 4: Minor issues, no mobility impact
- Priority 5 (lowest): Cosmetic only, no functional impact

In [11]:
# Define a system prompt for claim routing according to the provided ClaimRouting class
# TODO: Complete the prompt by replacing the parts marked with **********


class ClaimRouting(BaseModel):
    claim_id: str
    queue: Literal["glass", "fast_track", "material_damage", "total_loss"]
    priority: int = Field(..., ge=1, le=5)


queue_routing_system_prompt = """
You are an auto insurance claim routing system. Your task is to determine the appropriate processing queue for each claim.

INPUT: a JSON object with two keys:
- "claim_info": an object with claim fields (including `loss_desc` and `damage_area`).
- "severity_assessment": an object with keys `severity` ("Minor"/"Moderate"/"Major") and `est_cost` (numeric).

OUTPUT RULES (strict):
- Return exactly one valid JSON object parsable by python `json.loads()` and nothing else.
- The JSON must contain only these keys: "claim_id", "queue", "priority".
  - "claim_id": string (use the input claim_id)
  - "queue": one of the exact strings: "glass", "fast_track", "material_damage", "total_loss"
  - "priority": integer from 1 (highest) to 5 (lowest)
- Do NOT include any explanatory text, Markdown, or additional fields.

ROUTING RULES:
- If severity == "Major": queue = "total_loss".
- Else if severity == "Moderate": queue = "material_damage".
- Else if severity == "Minor":
  - If `damage_area` contains ONLY glass-related areas (e.g., "windshield", "glass", "window") and no other areas, queue = "glass".
  - Otherwise queue = "fast_track".

PRIORITY GUIDELINES (choose the single most appropriate integer):
- 1: Immediate safety issues or serious injuries (e.g., passengers transported to hospital), vehicle not drivable and customer stranded, airbags deployed with injuries.
- 2: Significant but contained damage affecting safety-related components (radiator, headlights) or multiple-panel moderate damage; customer may be at risk or reduced mobility.
- 3: Standard moderate claims without immediate safety risk.
- 4: Minor damage with some functional impact but not safety-critical (limited mobility impact).
- 5: Cosmetic-only minor damage (scratches, keying) or small glass chips with no mobility or safety impact.

When assessing priority, prefer safety-first: if conflicting signals exist, choose the higher priority (lower integer) that addresses safety or injury.

Examples of valid outputs:
{"claim_id": "C010", "queue": "total_loss", "priority": 1}
{"claim_id": "C001", "queue": "glass", "priority": 5}
{"claim_id": "C002", "queue": "material_damage", "priority": 3}

Only respond with the JSON object, nothing else.
"""

In [12]:
# Define a gate check function and assess_severity function
# No updates needed in this cell


def gate3_validate_routing(routing_json: str) -> ClaimRouting:
    """
    Gate 3: Validates that the claim is routed to a valid queue.
    Returns validated ClaimRouting object or raises validation error.
    """
    try:
        # Parse the JSON string
        routing_dict = json.loads(routing_json)
        # Validate with Pydantic model
        validated_routing = ClaimRouting(**routing_dict)
        return validated_routing
    except Exception as e:
        raise ValueError(f"Gate 3 validation failed: {str(e)}")


def route_claim(
    claim_info: ClaimInformation, severity_assessment: Optional[SeverityAssessment]
) -> Optional[ClaimRouting]:
    """
    Stage 3: Route claim to appropriate queue
    """
    if severity_assessment is None:
        return None

    # Create input for the routing model
    routing_input = {
        "claim_info": claim_info.model_dump(),
        "severity_assessment": severity_assessment.model_dump(),
    }

    messages = [
        {"role": "system", "content": queue_routing_system_prompt},
        {"role": "user", "content": json.dumps(routing_input)},
    ]

    response = get_completion(messages=messages)

    # Gate check: validate the routing decision
    try:
        validated_routing = gate3_validate_routing(response)
        return validated_routing
    except ValueError as e:
        print(f"Gate 3 failed: {e}. Response: {response}")
        return None

In [13]:
# Run the routing function on the sample data
# No updates needed in this cell

routed_claim_items = [
    route_claim(claim, severity_assessment)
    for claim, severity_assessment in zip(
        extracted_claim_info_items, severity_assessment_items
    )
]

routed_claim_items

[ClaimRouting(claim_id='C001', queue='glass', priority=5),
 ClaimRouting(claim_id='C002', queue='material_damage', priority=3),
 ClaimRouting(claim_id='C003', queue='total_loss', priority=1),
 ClaimRouting(claim_id='C004', queue='material_damage', priority=3),
 ClaimRouting(claim_id='C005', queue='material_damage', priority=3),
 ClaimRouting(claim_id='C006', queue='material_damage', priority=3),
 ClaimRouting(claim_id='C007', queue='total_loss', priority=1),
 ClaimRouting(claim_id='C008', queue='fast_track', priority=5),
 ClaimRouting(claim_id='C009', queue='fast_track', priority=4),
 ClaimRouting(claim_id='C010', queue='total_loss', priority=1)]

## 7. Review Outputs

Let's put our data into a pandas dataframe for easier analysis.

In [14]:
# No updates needed in this cell

import pandas as pd

records = []
for claim, severity_assessment, routed_claim in zip(
    extracted_claim_info_items, severity_assessment_items, routed_claim_items
):
    record = {}
    record.update(claim)
    record.update(severity_assessment)
    record.update(routed_claim)
    records.append(record)


# Show the entire dataframe since it is not too large
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(records)

df

,claim_id,name,vehicle,loss_desc,damage_area,severity,est_cost,queue,priority
0,C001,John Smith,2018 Toyota Camry,"While driving on the highway, a rock hit my windshield and caused a small chip about the size of a quarter. No other damage was observed.",[windshield],Minor,150.0,glass,5
1,C002,Sarah Johnson,2020 Honda Civic,"Someone hit the parked car, denting the rear bumper and breaking the taillight.","[rear, bumper, glass]",Moderate,2500.0,material_damage,3
2,C003,Michael Rodriguez,2022 Ford F-150,"Serious collision at an intersection with severe front-end damage, including hood, bumper, radiator, and engine compartment. Airbags deployed and vehicle is not drivable.","[front, hood, bumper]",Major,15000.0,total_loss,1
3,C004,Emma Williams,2019 Subaru Outback,"My car was damaged in a hailstorm. There are multiple dents on the hood, roof, and trunk. The side mirrors were also damaged and one window has a small crack.","[hood, roof, trunk, side, glass]",Moderate,2500.0,material_damage,3
4,C005,David Brown,2021 Tesla Model 3,Someone keyed my car in the parking lot. There are deep scratches along both doors on the driver's side.,[door],Moderate,2500.0,material_damage,3
5,C006,Lisa Davis,2017 BMW X5,"Rear-ended at a stoplight. Damage includes bumper, trunk, rear lights, and shattered rear windshield.","[bumper, trunk, rear, glass]",Moderate,3500.0,material_damage,3
6,C007,James Wilson,2016 Audi A4,"Hit a deer causing damage to the front including bumper, grille, headlights, hood, and radiator.","[front, bumper, hood, glass]",Major,15000.0,total_loss,1
7,C008,Ana Pereira,2015 Hyundai Elantra,"Lightly hit the car behind while backing out of a parking space, causing a small dent on the rear bumper and a scuff on the paint.","[bumper, rear]",Minor,300.0,fast_track,5
8,C009,Bruno Silva,2012 Volkswagen Golf,clipped a curb and scraped the right front quarter panel with a dent and paint damage,[quarter panel],Minor,450.0,fast_track,4
9,C010,Carla Mendes,2019 Chevrolet Silverado,"High-speed multi-vehicle collision causing heavy damage to the front of the truck, airbags deployed, and passengers injured.",[front],Major,15000.0,total_loss,1


## 8. Reflection & Transfer
Reflect on the effectiveness of chaining prompts for this task:

1. Prompt Chain Architecture:
* How does breaking down the task into stages affect the performance?
* What are the benefits of having gate checks between stages?
* How could the chain design be improved?

2. Error Handling:
* What types of errors might occur at each stage?
* How could we make the chain more robust against errors?
* What would a good fallback strategy look like?

3. Scalability:
* How well would this approach scale to handle more complex claims?
* What challenges might arise when processing thousands of claims?
* How could the prompt chain be optimized for efficiency?

4. Transfer to Other Domains:
* How could this prompt chaining approach be applied to other domains?
* What general principles of prompt chaining can we extract from this exercise?

## Summary

🎉 Congratulations! 🎉 You've built an impressive prompt chain system for insurance claims!
You transformed messy FNOL text into structured data, assessed damage severity, and routed claims to the right queues, all with robust gate checks! 🚀✨

Remember:

- 🔗 Chained prompts break complex tasks into manageable steps
- 🛡️ Gate checks prevent error cascades
- 🧠 Having specialized prompts helps keep code focused and maintainable

You've mastered a powerful pattern for countless business processes! 🏆
Amazing work on your agentic reasoning system! 💯